In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt
import pickle

%matplotlib inline

In [ ]:
"""
def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url))
    return pd.concat(dfs)
        
week_nums = [160903, 160910, 160917]
turnstiles_df = get_data(week_nums)
""""

In [3]:
path = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_190921.txt'
col_names= ['CA','UNIT','SCP','STATION','LINENAME','DIVISION','DATE','TIME','DESC','ENTRIES','EXITS']
df = pd.read_csv(path, sep=",", header=0, names = col_names, parse_dates=[['DATE', 'TIME']])

In [4]:
df.head()

,DATE_TIME,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DESC,ENTRIES,EXITS
0,2019-09-14 00:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7198818,2438323
1,2019-09-14 04:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7198834,2438325
2,2019-09-14 08:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7198847,2438354
3,2019-09-14 12:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7198929,2438428
4,2019-09-14 16:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7199125,2438483


In [5]:
df.columns

Index(['DATE_TIME', 'CA', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

In [12]:
df.groupby(["CA", "UNIT", "SCP", "STATION", "DATE_TIME"]).ENTRIES.count().reset_index().sort_values("ENTRIES", ascending=False).head(5)

,CA,UNIT,SCP,STATION,DATE_TIME,ENTRIES
127639,R101,R001,02-00-04,SOUTH FERRY,2019-09-17 05:00:00,2
127765,R101,R001,02-00-07,SOUTH FERRY,2019-09-17 05:00:00,2
136609,R138,R293,00-03-02,34 ST-PENN STA,2019-09-18 14:00:00,1
136611,R138,R293,00-03-02,34 ST-PENN STA,2019-09-18 22:00:00,1
136612,R138,R293,00-03-02,34 ST-PENN STA,2019-09-19 02:00:00,1


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204928 entries, 0 to 204927
Data columns (total 10 columns):
DATE_TIME    204928 non-null datetime64[ns]
CA           204928 non-null object
UNIT         204928 non-null object
SCP          204928 non-null object
STATION      204928 non-null object
LINENAME     204928 non-null object
DIVISION     204928 non-null object
DESC         204928 non-null object
ENTRIES      204928 non-null int64
EXITS        204928 non-null int64
dtypes: datetime64[ns](1), int64(2), object(7)
memory usage: 15.6+ MB


In [44]:
mask = ((df["CA"] == "R101") & 
(df["UNIT"] == "R001") & 
(df["SCP"] == "02-00-04") &
(df["STATION"] == "SOUTH FERRY") & 
(df['DATE_TIME'] == '2019-09-17 05:00:00'))
# (df["DATE_TIME"].dt.date == datetime.datetime(2019, 9, 17).date()))

In [45]:
df[mask].head()

,DATE_TIME,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DESC,ENTRIES,EXITS
127639,2019-09-17 05:00:00,R101,R001,02-00-04,SOUTH FERRY,1RW,IRT,REGULAR,3062453,2464020
127640,2019-09-17 05:00:00,R101,R001,02-00-04,SOUTH FERRY,1RW,IRT,RECOVR AUD,3062452,2464020


In [46]:
df = df[df.DESC == 'REGULAR']

In [48]:
df.groupby(["CA", "UNIT", "SCP", "STATION", "DATE_TIME"]).ENTRIES.count().reset_index().sort_values("ENTRIES", ascending=False).head(5)

,CA,UNIT,SCP,STATION,DATE_TIME,ENTRIES
0,A002,R051,02-00-00,59 ST,2019-09-14 00:00:00,1
136149,R138,R293,00-03-02,34 ST-PENN STA,2019-09-18 02:00:00,1
136151,R138,R293,00-03-02,34 ST-PENN STA,2019-09-18 10:00:00,1
136152,R138,R293,00-03-02,34 ST-PENN STA,2019-09-18 14:00:00,1
136153,R138,R293,00-03-02,34 ST-PENN STA,2019-09-18 18:00:00,1


In [51]:
daily_df = df.groupby(["CA", "UNIT", "SCP", "STATION", "DATE_TIME"]).ENTRIES.first().reset_index()

In [52]:
daily_df.head()

,CA,UNIT,SCP,STATION,DATE_TIME,ENTRIES
0,A002,R051,02-00-00,59 ST,2019-09-14 00:00:00,7198818
1,A002,R051,02-00-00,59 ST,2019-09-14 04:00:00,7198834
2,A002,R051,02-00-00,59 ST,2019-09-14 08:00:00,7198847
3,A002,R051,02-00-00,59 ST,2019-09-14 12:00:00,7198929
4,A002,R051,02-00-00,59 ST,2019-09-14 16:00:00,7199125


In [54]:
daily_df[["PREV_DATE", "PREV_ENTRIES"]] = (daily_df
                                                       .groupby(["CA", "UNIT", "SCP", "STATION"])["DATE_TIME", "ENTRIES"]
                                                       .transform(lambda grp: grp.shift(1)))

In [55]:
daily_df.head()

,CA,UNIT,SCP,STATION,DATE_TIME,ENTRIES,PREV_DATE,PREV_ENTRIES
0,A002,R051,02-00-00,59 ST,2019-09-14 00:00:00,7198818,NaT,NaN
1,A002,R051,02-00-00,59 ST,2019-09-14 04:00:00,7198834,2019-09-14 00:00:00,7198818.0
2,A002,R051,02-00-00,59 ST,2019-09-14 08:00:00,7198847,2019-09-14 04:00:00,7198834.0
3,A002,R051,02-00-00,59 ST,2019-09-14 12:00:00,7198929,2019-09-14 08:00:00,7198847.0
4,A002,R051,02-00-00,59 ST,2019-09-14 16:00:00,7199125,2019-09-14 12:00:00,7198929.0


In [72]:
daily_df.dropna(subset=["PREV_DATE"], axis=0, inplace=True)

In [73]:
daily_df[daily_df["ENTRIES"] < daily_df["PREV_ENTRIES"]].head(20)


,CA,UNIT,SCP,STATION,DATE_TIME,ENTRIES,PREV_DATE,PREV_ENTRIES
1601,A011,R080,01-03-00,57 ST-7 AV,2019-09-14 04:00:00,885813434,2019-09-14 00:00:00,885813480.0
1602,A011,R080,01-03-00,57 ST-7 AV,2019-09-14 08:00:00,885813415,2019-09-14 04:00:00,885813434.0
1603,A011,R080,01-03-00,57 ST-7 AV,2019-09-14 12:00:00,885813364,2019-09-14 08:00:00,885813415.0
1604,A011,R080,01-03-00,57 ST-7 AV,2019-09-14 16:00:00,885813254,2019-09-14 12:00:00,885813364.0
1605,A011,R080,01-03-00,57 ST-7 AV,2019-09-14 20:00:00,885813143,2019-09-14 16:00:00,885813254.0
1606,A011,R080,01-03-00,57 ST-7 AV,2019-09-15 00:00:00,885813037,2019-09-14 20:00:00,885813143.0
1607,A011,R080,01-03-00,57 ST-7 AV,2019-09-15 04:00:00,885812998,2019-09-15 00:00:00,885813037.0
1608,A011,R080,01-03-00,57 ST-7 AV,2019-09-15 08:00:00,885812978,2019-09-15 04:00:00,885812998.0
1609,A011,R080,01-03-00,57 ST-7 AV,2019-09-15 12:00:00,885812938,2019-09-15 08:00:00,885812978.0
1610,A011,R080,01-03-00,57 ST-7 AV,2019-09-15 16:00:00,885812843,2019-09-15 12:00:00,885812938.0


In [74]:
new_mask = ((df["CA"] == "A011") & 
(df["UNIT"] == "R080") & 
(df["SCP"] == "01-03-00") & 
(df["STATION"] == "57 ST-7 AV") &
(df["DATE_TIME"] == '2019-09-14 04:00:00'))

In [75]:
df[new_mask].head()

,DATE_TIME,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DESC,ENTRIES,EXITS
1601,2019-09-14 04:00:00,A011,R080,01-03-00,57 ST-7 AV,NQRW,BMT,REGULAR,885813434,490308745


In [76]:
(daily_df[daily_df["ENTRIES"] < daily_df["PREV_ENTRIES"]]
    .groupby(["CA", "UNIT", "SCP", "STATION"])
    .size())

CA     UNIT  SCP       STATION        
A011   R080  01-03-00  57 ST-7 AV         41
A025   R023  01-03-01  34 ST-HERALD SQ    41
A049   R088  02-05-00  CORTLANDT ST       12
A066   R118  00-00-00  CANAL ST           40
B028   R136  01-00-01  SHEEPSHEAD BAY      1
C008   R099  00-00-00  DEKALB AV          41
D005   R398  00-06-00  NEW UTRECHT AV      1
D011   R394  01-06-00  BAY PKWY           36
             01-06-01  BAY PKWY            8
H003   R163  01-00-02  6 AV               41
H022   R279  00-06-01  JEFFERSON ST        1
H023   R236  00-06-00  DEKALB AV          29
J017   R432  00-00-02  CHAUNCEY ST         1
J034   R007  00-00-02  104 ST             41
JFK01  R535  00-00-03  HOWARD BCH JFK      1
K026   R100  00-00-01  METROPOLITAN AV    41
             00-00-04  METROPOLITAN AV     1
N006A  R280  00-00-00  190 ST             41
N025   R102  01-00-00  125 ST              1
N063A  R011  00-00-04  42 ST-PORT AUTH    41
             00-00-05  42 ST-PORT AUTH    41
             00-

In [77]:
def get_daily_counts(row, max_counter):
    counter = row["ENTRIES"] - row["PREV_ENTRIES"]
    if counter < 0:
        counter = -counter
    if counter > max_counter:
        print(row["ENTRIES"], row["PREV_ENTRIES"])
        return 0
    return counter

In [78]:
daily_df["DAILY_ENTRIES"] = daily_df.apply(get_daily_counts, axis=1, max_counter=1000000) 

875479771 775219651.0
590061 9157505.0
98 1479461.0
126 6053924.0
24 2781186.0
1507350 4609.0
33600558 189253.0
507 3676614.0


In [79]:
daily_df.head()

,CA,UNIT,SCP,STATION,DATE_TIME,ENTRIES,PREV_DATE,PREV_ENTRIES,DAILY_ENTRIES
1,A002,R051,02-00-00,59 ST,2019-09-14 04:00:00,7198834,2019-09-14 00:00:00,7198818.0,16.0
2,A002,R051,02-00-00,59 ST,2019-09-14 08:00:00,7198847,2019-09-14 04:00:00,7198834.0,13.0
3,A002,R051,02-00-00,59 ST,2019-09-14 12:00:00,7198929,2019-09-14 08:00:00,7198847.0,82.0
4,A002,R051,02-00-00,59 ST,2019-09-14 16:00:00,7199125,2019-09-14 12:00:00,7198929.0,196.0
5,A002,R051,02-00-00,59 ST,2019-09-14 20:00:00,7199405,2019-09-14 16:00:00,7199125.0,280.0


In [86]:
daily_df.groupby(['STATION'])['DAILY_ENTRIES'].sum().sort_values(ascending = False).head(20)

STATION
GRD CNTRL-42 ST    1104009.0
34 ST-PENN STA     1079216.0
METROPOLITAN AV     863264.0
34 ST-HERALD SQ     761408.0
23 ST               741078.0
BOWLING GREEN       716792.0
14 ST-UNION SQ      654461.0
42 ST-PORT AUTH     641627.0
TIMES SQ-42 ST      625091.0
FULTON ST           597076.0
86 ST               543440.0
RIT-MANHATTAN       517985.0
125 ST              514489.0
CANAL ST            511450.0
JUNCTION BLVD       482320.0
59 ST COLUMBUS      470143.0
THIRTY THIRD ST     425550.0
14 ST               405175.0
96 ST               402275.0
GROVE STREET        389685.0
Name: DAILY_ENTRIES, dtype: float64